Importing my functions.py file

In [1]:
from functions import *

[nltk_data] Downloading package punkt to /Users/saifdev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/saifdev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


First, I will load all (approximately) 6000 tsv files into one final dataset.

In [2]:
directory = "/Users/saifdev/Desktop/files_tsv"
output_file = "/Users/saifdev/Desktop/grande.tsv"
dataset = create_store_tsv(directory, output_file)

In [3]:
dataset.head(5)

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration
0,"Masters of Science in Business, Supply Chain A...",Oregon State University,School of Business,Full time&Part time,Master of Science in Business (MSB)Our Master ...,See Course,You can find more information here:Please see ...,MSc,12 months,Corvallis,USA,Online
1,Immersive Technology (with Advanced Practice) MSc,Teesside University,"School of Computing, Engineering & Design Tech...",Full time,Immersive technologies is a rapidly growing se...,"January, September",Please see the university website for further ...,MSc,2 Years,Middlesbrough,United Kingdom,On Campus
2,Clinical Neuropsychology - MSc(MedSci)/PgDip,University of Glasgow,"College of Medical, Veterinary & Life Sciences",Full time&Part time,Designed for those who are eligible for regist...,September,Please see the university website for further ...,"PGDip,MSc",See Programme Description,Glasgow,United Kingdom,On Campus
3,Demography and Social Inequality DDM,University of Groningen,Spatial Sciences,Full time,What impact did the recent economic crisis hav...,September,Please see the university website for further ...,MSc,2 years full time,Groningen,Netherlands,On Campus
4,MSc Financial Mathematics,University College Dublin,UCD School of Mathematics and Statistics,Full time&Part time,"Graduate Taught (level 9 nfq, credits 90)The M...",September,Please see the university website for further ...,MSc,1 year full time - 2 years part time,Dublin,Ireland,On Campus


Now, I will make a copy to protect the original dataset

In [4]:
df = dataset.copy()
df.head(3)

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration
0,"Masters of Science in Business, Supply Chain A...",Oregon State University,School of Business,Full time&Part time,Master of Science in Business (MSB)Our Master ...,See Course,You can find more information here:Please see ...,MSc,12 months,Corvallis,USA,Online
1,Immersive Technology (with Advanced Practice) MSc,Teesside University,"School of Computing, Engineering & Design Tech...",Full time,Immersive technologies is a rapidly growing se...,"January, September",Please see the university website for further ...,MSc,2 Years,Middlesbrough,United Kingdom,On Campus
2,Clinical Neuropsychology - MSc(MedSci)/PgDip,University of Glasgow,"College of Medical, Veterinary & Life Sciences",Full time&Part time,Designed for those who are eligible for regist...,September,Please see the university website for further ...,"PGDip,MSc",See Programme Description,Glasgow,United Kingdom,On Campus


# 2. Search Engine
### 2.0.0 Preprocessing the text
First, you must pre-process all the information collected for each MSc by:

  * Removing stopwords
  * Removing punctuation
  * Stemming
  * Anything else you think it's needed


Now, as the dataset is loaded, I will apply some preprocessing to all columns. This pre-processing includes, tokenizing, standardazing the text (lower case and removing stop words), and stemming the text of all columns.

In [5]:
desired_columns = [x for x in df.columns if x != "fees"]
for column in desired_columns:
    df[column+"_clean"] = df[column].apply(lambda x: preprocessing(x))

In [6]:
df.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,...,universityName_clean,facultyName_clean,isItFullTime_clean,description_clean,startDate_clean,modality_clean,duration_clean,city_clean,country_clean,administration_clean
0,"Masters of Science in Business, Supply Chain A...",Oregon State University,School of Business,Full time&Part time,Master of Science in Business (MSB)Our Master ...,See Course,You can find more information here:Please see ...,MSc,12 months,Corvallis,...,"[oregon, state, univers]","[school, busi]","[full, time, part, time]","[master, scienc, busi, msb, master, scienc, bu...","[see, cours]",[msc],[month],[corval],[usa],[onlin]
1,Immersive Technology (with Advanced Practice) MSc,Teesside University,"School of Computing, Engineering & Design Tech...",Full time,Immersive technologies is a rapidly growing se...,"January, September",Please see the university website for further ...,MSc,2 Years,Middlesbrough,...,"[teessid, univers]","[school, comput, engin, design, technolog]","[full, time]","[immers, technolog, rapidli, grow, sector, inv...","[januari, septemb]",[msc],[year],[middlesbrough],"[unit, kingdom]",[campu]
2,Clinical Neuropsychology - MSc(MedSci)/PgDip,University of Glasgow,"College of Medical, Veterinary & Life Sciences",Full time&Part time,Designed for those who are eligible for regist...,September,Please see the university website for further ...,"PGDip,MSc",See Programme Description,Glasgow,...,"[univers, glasgow]","[colleg, medic, veterinari, life, scienc]","[full, time, part, time]","[design, elig, registr, health, profess, counc...",[septemb],"[pgdip, msc]","[see, programm, descript]",[glasgow],"[unit, kingdom]",[campu]
3,Demography and Social Inequality DDM,University of Groningen,Spatial Sciences,Full time,What impact did the recent economic crisis hav...,September,Please see the university website for further ...,MSc,2 years full time,Groningen,...,"[univers, groningen]","[spatial, scienc]","[full, time]","[impact, recent, econom, crisi, live, work, co...",[septemb],[msc],"[year, full, time]",[groningen],[netherland],[campu]
4,MSc Financial Mathematics,University College Dublin,UCD School of Mathematics and Statistics,Full time&Part time,"Graduate Taught (level 9 nfq, credits 90)The M...",September,Please see the university website for further ...,MSc,1 year full time - 2 years part time,Dublin,...,"[univers, colleg, dublin]","[ucd, school, mathemat, statist]","[full, time, part, time]","[graduat, taught, level, nfq, credit, msc, fin...",[septemb],[msc],"[year, full, time, year, part, time]",[dublin],[ireland],[campu]


Now, it is time to do some preprocssing with the fees column. We have to ensure that all the fees are in Euro (decided) and the column is of dtype float. So, let's do it!

First, I will replace all the entries that are prompting to contact the university or visit webpage. Then, I will see what currency symbols have been used through out the dataset so that I can desing my regex accordingly.

In [7]:
if_website_or_contact_replace(df, "fees")
df["fees_clean"] = df["fees_clean"].apply(lambda x: "Not Available" if pd.isnull(x) else x)
symbols_list = extract_symbols(df)
symbols_list

['$', '€', '£']

Now, from all of the text in each row of the "fees" column, I will only keep the max amount and the currency name.

In [8]:
df["fees_clean"] = df["fees_clean"].apply(lambda x: extract_numeric_and_currency(x))
df.loc[:, ["fees","fees_clean"]]

,fees,fees_clean
0,You can find more information here:Please see ...,Not Available
1,Please see the university website for further ...,Not Available
2,Please see the university website for further ...,Not Available
3,Please see the university website for further ...,Not Available
4,Please see the university website for further ...,Not Available
...,...,...
5974,"Full time -£14,100Part time - £7,050",14100.0 GBP
5975,Tuition fees (2024-2025):EU/EEA nationals: 835...,2782.0 EUR
5976,"UK studentsFull-time students: £9,600Part-time...",9600.0 GBP
5977,Cost: 28020 USDThe program lasts 2 years (4 se...,28020.0 USD


Now, I will create two more columns and use these columns to make make a new column FEE_EUR where all the fees are in EUR.

In [9]:
# Apply extraction functions to create new columns
df['numeric_value'] = df['fees_clean'].apply(extract_numeric_value)
df['currency_name'] = df['fees_clean'].apply(extract_currency_name)

In [ ]:
tqdm.pandas()
def apply_convert_to_eur(row):
    return convert_to_eur(row['numeric_value'], row['currency_name'])
df['FEE_EUR'] = df.progress_apply(apply_convert_to_eur, axis=1)

Finally, I will delete the extra columns. Namely: "fees_clean", "numeric_value", "currency_name"

In [ ]:
df.columns
columns_to_drop = ["fees_clean", "numeric_value", "currency_name"]
df = df.drop(columns=columns_to_drop)

### 2.1. Conjunctive query

2.1.1 Create your index!
Before building the index,

Create a file named vocabulary, in the format you prefer, that maps each word to an integer (term_id).

In [10]:
vocabulary = set() # set to remove duplicates and efficiency
# I will only used the description column as stated in the HW
df.description_clean.apply(lambda row: [vocabulary.add(word) for word in row])
vocabulary = list(vocabulary)

The variable vocabulary contains, in a list, all the words used in the description column. The question requires us to make a file named vocabulary that *maps each word to an integer (term_id).*

In [11]:
# maps each word to an integer (term_id)
vocab_with_index = dict()
unique_id = 0
for word in vocabulary:
  vocab_with_index[word] = unique_id
  unique_id+=1
vocab_with_index

with open("/Users/saifdev/Desktop/vocabulary.json", "w") as file: # storing for reusability
    json.dump(vocab_with_index, file)

In [12]:
vocab_with_index = json.load(open("/Users/saifdev/Desktop/vocabulary.json", "r")) # loading for use

#### 2.1.1
Then, the first brick of your homework is to create the Inverted Index. It will be a dictionary in this format:

{
term_id_1:[document_1, document_2, document_4],
term_id_2:[document_1, document_3, document_5, document_6],
...}

To create the inverted index, I can use the procedure used in the lab as follows:

In [13]:
Terms = pd.DataFrame(data=vocab_with_index.keys(), columns=['Term']) # from lab
tqdm.pandas() # to create partial inverted index, as shown in lab
Terms['reverse'] = Terms.Term.progress_apply(lambda item: list(df.loc[df.description_clean.apply(lambda row: item in row)].index))
Terms.head()
# inverted_index = dict(zip(Terms.index, Terms['reverse'])) 

100%|██████████| 9707/9707 [00:43<00:00, 225.54it/s]


,Term,reverse
0,length,"[151, 272, 363, 430, 657, 852, 1185, 1195, 131..."
1,besançon,[2535]
2,astrobiolog,"[3857, 5067]"
3,bypass,[1496]
4,programmeprovid,[782]


OR, I can use the function that I create to make an inverted list. Here's how:

In [14]:
inverted_index = inverted_index(vocab_with_index, df)

I will now store this inverted index locally

In [15]:
with open("/Users/saifdev/Desktop/inverted_index_1.json", "w") as file: # storing for reusability
    json.dump(inverted_index, file)
inverted_index = json.load(open("/Users/saifdev/Desktop/inverted_index_1.json", "r")) # loading for use

2.1.2) Execute the query
Given a query input by the user, for example:

advanced knowledge

The Search Engine is supposed to return a list of documents.

For this task, I will first take the input from the user and then normalize the query using the same technique I used to preprocess the description column.

In [16]:
user_input = input("Add your Input")
normalized_query = normalize_query(user_input)

Now, I will extract the term_id from the vocabulary with the index in order to utilize it for the inverted index

In [17]:
term_ids = get_term_id_from_query(vocab_with_index, normalized_query)

In [18]:
new_dataset = search_engine(term_ids, df, inverted_index).head(10)
new_dataset

,courseName,universityName,description
0,Biomedical Science (Medical Immunology) MSc,This master's programme will allow you to deve...,This master's programme will allow you to deve...
1,MSc In People Analytics,Start date: January 2024Play an essential role...,Start date: January 2024Play an essential role...
2,Clinical Pharmacy - MSc,The Clinical Pharmacy MSc has been designed to...,The Clinical Pharmacy MSc has been designed to...
3,Master of Science (MS) in Toxicology for Human...,The Master of Science (MS) in Toxicology for H...,The Master of Science (MS) in Toxicology for H...
4,Environmental Management for Business (MSc),Study for a successful environmental managemen...,Study for a successful environmental managemen...
5,Engineering Management MSc,The career of an engineering project manager i...,The career of an engineering project manager i...
6,Electronics Engineering,This programme focuses on the design of integr...,This programme focuses on the design of integr...
7,MSc - Economics,Our MSc programmes in Economics will give you ...,Our MSc programmes in Economics will give you ...
8,"Data, Inequality and Society (Online Learning)...",Programme descriptionWhat makes societies incl...,Programme descriptionWhat makes societies incl...
9,Big Data Technologies MSc,This programme is ideal if you want to pursue ...,This programme is ideal if you want to pursue ...


#### 2.2) Conjunctive query & Ranking score

For the second search engine, given a query, we want to get the top-k (the choice of k it's up to you!) documents related to the query. In particular:

Find all the documents that contain all the words in the query.
Sort them by their similarity with the query.
Return in output k documents, or all the documents with non-zero similarity with the query when the results are less than k. You must use a heap data structure (you can use Python libraries) for maintaining the top-k documents.
To solve this task, you must use the tfIdf score and the Cosine similarity. The field to consider is still the description. Let's see how.

##### 2.2.1) Inverted index
Your second Inverted Index must be of this format:

{
term_id_1:[(document1, tfIdf_{term,document1}), (document2, tfIdf_{term,document2}), (document4, tfIdf_{term,document4}), ...],
term_id_2:[(document1, tfIdf_{term,document1}), (document3, tfIdf_{term,document3}), (document5, tfIdf_{term,document5}), (document6, tfIdf_{term,document6}), ...],
...}



In [ ]:
word_and_appearances = dict(zip(Terms["Term"], Terms['reverse'])) # will help with the search engine

In [ ]:
def inverted_index2(word_and_appearances, df):
    inverted_ind = {}
    words = list(word_and_appearances.keys())
    for key in word_and_appearances.keys():
        appearances = word_and_appearances[key]
        result = []
        for doc in appearances:
            interested_row = df.at[doc, "description_clean"] # extract the list of tokens from a proper column
            interested_word = key #i-th word
            tf = interested_row.count(interested_word) / len(interested_row)
            idf = math.log(len(df)/len(appearances))
            tf_idf = round(tf * idf, 3)
            result.append((doc,tf_idf))

        inverted_ind[words.index(key)] = result # insert the result into the inverted list

    return inverted_ind

In [ ]:
inv_lst2 = inverted_list_2(vocab_with_index, df)

In [ ]:
def create_documents_list(df, inv_lst2, word_and_appearances):
    ### Input == The clean dataset, the new inverted list and the vocabulary
    ### Output == A list of dictionary defined documents. For each row in clean dataset i create a dictionary and in this dictionary
    ### I define like keys the token of Plot. For each token (keys) i use like values the tfidf.
    documents = []
    indexes = list(word_and_appearances.keys())
    for i, row in df.iterrows():
        tokens = {}
        for token in df.at[i, "description_clean"]:
            tuple_list_values = inv_lst2[indexes.index(token)]
            for x in tuple_list_values:
                if x[0] == i:
                    tokens[token] = x[1]  
                    break
        documents.append(tokens)
    return documents

In [ ]:
documents = create_documents_list(df, inv_lst2, word_and_appearances)

In [ ]:
word_and_appearances